# CA Wildfire Causes Analysis

The first module of this notebook allows you to analyze the number of acres burned by each of the different causes defined by Cal Fire between 1887 and 2020. 

The second module of this notebook allows you to analyze the same data in a shorter period of time: from 2000 to 2020.


### Data description
* Data source: Fire Perimeters through 2020, CALFIRE (Updated Apr. 2021, url: https://frap.fire.ca.gov/mapping/gis-data/)
* Time range: 1887 - 2020
* Total values: 21318
* Missing values: 77 records missing 'year_' value (removed for analysis)
* Duplicate values:

### Data limitations
* Unknown causes: Almost half of the records (over 9,400) in the database are listed with unknown cause. I suspect that this database is not updated often with the results of fire investigations. After comparing yearly totals of the powerline database CALFIRE provided, I noted a difference in fires caused by powerlines of hundreds of records per year.
* Missing values: 77 records do not have a year assigned and cannot be used for the analysis. Almost 1,000 incidents do not have an incident number, which makes merges inaccurate.
* After merging the database with the powerlines database, I suspect that this database is highly incomplete. The number of fires caused by powerlines in 2019 excedeed the total number of fires provided in this database.

### Configuration and Exploration:
Let's begin by importing the Python tools necessary for the analysis.

In [97]:
import pandas as pd
import altair as alt
import datetime

Import Cal Fire Dataset

In [98]:
ca_fires = pd.read_csv("data/raw/fires_data.csv") # Import database

In [99]:
len(ca_fires)

21318

In [100]:
# Rows where year in null
len(ca_fires[ca_fires.YEAR_.isna()])

77

In [101]:
# Rows where inc number in null
len(ca_fires[ca_fires.INC_NUM.isna()])

922

In [102]:
# Rows where inc number in null
len(ca_fires[ca_fires.INC_NUM.str.strip() == ''])

4

In [103]:
ca_fires = ca_fires[ca_fires.YEAR_.isna() == False] # Filtering database to only include rows where year is not null

In [104]:
ca_fires.YEAR_ = ca_fires.YEAR_.astype(int)

In [105]:
ca_fires.columns = ca_fires.columns.str.replace(' ', '_').str.lower()

### Analysis: 
The data source is now prepared for analysis.

In [106]:
ca_fires.info() # Explore database

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21241 entries, 0 to 21317
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   objectid      21241 non-null  int64  
 1   year_         21241 non-null  int64  
 2   state         21238 non-null  object 
 3   agency        21236 non-null  object 
 4   unit_id       21221 non-null  object 
 5   fire_name     21118 non-null  object 
 6   inc_num       20387 non-null  object 
 7   alarm_date    15954 non-null  object 
 8   cont_date     8638 non-null   object 
 9   cause         21193 non-null  object 
 10  comments      3996 non-null   object 
 11  report_ac     8699 non-null   float64
 12  gis_acres     21234 non-null  float64
 13  c_method      9028 non-null   object 
 14  objective     21046 non-null  object 
 15  fire_num      17304 non-null  object 
 16  shape_length  21241 non-null  float64
 17  shape_area    21241 non-null  float64
dtypes: float64(4), int64(2), o

### Explore:
We start to explore the different "CAUSES" included in our dataset.

In [107]:
ca_fires.head(3)

,objectid,year_,state,agency,unit_id,fire_name,inc_num,alarm_date,cont_date,cause,comments,report_ac,gis_acres,c_method,objective,fire_num,shape_length,shape_area
0,21440,2020,California,California Department of Forestry and Fire Pro...,Nevada - Yuba - Placer CAL FIRE,NELSON,13212,6/18/20 0:00,6/23/20 0:00,11 - Powerline,NaN,110.0,109.602501,1 - GPS Ground,Suppression (Wildfire),NaN,3252.523280,4.435447e+05
1,21441,2020,California,California Department of Forestry and Fire Pro...,Nevada - Yuba - Placer CAL FIRE,AMORUSO,11799,6/1/20 0:00,6/4/20 0:00,2 - Equipment Use,NaN,670.0,685.585022,1 - GPS Ground,Suppression (Wildfire),NaN,9653.760308,2.774464e+06
2,21442,2020,California,California Department of Forestry and Fire Pro...,Nevada - Yuba - Placer CAL FIRE,ATHENS,18493,8/10/20 0:00,3/1/20 0:00,14 - Unknown / Unidentified,NaN,26.0,27.300480,1 - GPS Ground,Suppression (Wildfire),NaN,1649.643235,1.104811e+05


In [108]:
ca_fires.cause.describe() # Explore "CAUSE"

count                           21193
unique                             18
top       14 - Unknown / Unidentified
freq                             9466
Name: cause, dtype: object

Filter down to number of wildfires due to each cause in California.

In [109]:
ca_fires.cause.value_counts() # Number of fires by "CAUSE"

14 - Unknown / Unidentified      9466
1 - Lightning                    3454
9 - Miscellaneous                3379
2 - Equipment Use                1246
7 - Arson                         903
5 - Debris                        723
10 - Vehicle                      454
11 - Powerline                    412
4 - Campfire                      380
3 - Smoking                       342
8 - Playing with fire             196
18 - Escaped Prescribed Burn       90
6 - Railroad                       80
15 - Structure                     21
19 - Illegal Alien Campfire        17
16 - Aircraft                      14
13 - Non-Firefighter Training      11
12 - Firefighter Training           5
Name: cause, dtype: int64

In [110]:
ca_fires.cause.value_counts(normalize=True)

14 - Unknown / Unidentified      0.446657
1 - Lightning                    0.162978
9 - Miscellaneous                0.159439
2 - Equipment Use                0.058793
7 - Arson                        0.042608
5 - Debris                       0.034115
10 - Vehicle                     0.021422
11 - Powerline                   0.019440
4 - Campfire                     0.017930
3 - Smoking                      0.016137
8 - Playing with fire            0.009248
18 - Escaped Prescribed Burn     0.004247
6 - Railroad                     0.003775
15 - Structure                   0.000991
19 - Illegal Alien Campfire      0.000802
16 - Aircraft                    0.000661
13 - Non-Firefighter Training    0.000519
12 - Firefighter Training        0.000236
Name: cause, dtype: float64

In [111]:
counts_by_cause = ca_fires.cause.value_counts().reset_index() # Reformat table
counts_by_cause.columns = ['cause', 'number_of_fires']
counts_by_cause

,cause,number_of_fires
0,14 - Unknown / Unidentified,9466
1,1 - Lightning,3454
2,9 - Miscellaneous,3379
3,2 - Equipment Use,1246
4,7 - Arson,903
5,5 - Debris,723
6,10 - Vehicle,454
7,11 - Powerline,412
8,4 - Campfire,380
9,3 - Smoking,342


In [112]:
fires_per_year = ca_fires.groupby('year_').objectid.count().reset_index().rename(columns={'objectid': 'fires_per_year'})
fires_per_year

,year_,fires_per_year
0,1878,1
1,1895,1
2,1896,1
3,1898,5
4,1900,12
...,...,...
118,2016,352
119,2017,607
120,2018,413
121,2019,312


### Filter:
You can run this filter with every cause.

In [113]:
my_cause = "11 - Powerline" # Filter by "CAUSE" - This filter can be use with any "CAUSE".

In [114]:
my_cause = ca_fires[ca_fires.cause == my_cause] # Filter by "my chosen cause".

In [115]:
my_cause.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412 entries, 0 to 21302
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   objectid      412 non-null    int64  
 1   year_         412 non-null    int64  
 2   state         412 non-null    object 
 3   agency        412 non-null    object 
 4   unit_id       410 non-null    object 
 5   fire_name     411 non-null    object 
 6   inc_num       404 non-null    object 
 7   alarm_date    402 non-null    object 
 8   cont_date     357 non-null    object 
 9   cause         412 non-null    object 
 10  comments      88 non-null     object 
 11  report_ac     333 non-null    float64
 12  gis_acres     412 non-null    float64
 13  c_method      361 non-null    object 
 14  objective     410 non-null    object 
 15  fire_num      196 non-null    object 
 16  shape_length  412 non-null    float64
 17  shape_area    412 non-null    float64
dtypes: float64(4), int64(2), obj

In [116]:
my_cause.head(3)

,objectid,year_,state,agency,unit_id,fire_name,inc_num,alarm_date,cont_date,cause,comments,report_ac,gis_acres,c_method,objective,fire_num,shape_length,shape_area
0,21440,2020,California,California Department of Forestry and Fire Pro...,Nevada - Yuba - Placer CAL FIRE,NELSON,13212,6/18/20 0:00,6/23/20 0:00,11 - Powerline,NaN,110.0,109.602501,1 - GPS Ground,Suppression (Wildfire),NaN,3252.523280,4.435447e+05
14,21454,2020,California,Department of Defense,NaN,PAVE PAWS,17717,8/2/20 0:00,8/2/20 0:00,11 - Powerline,Beale Air Force Base,532.0,532.773377,1 - GPS Ground,Suppression (Wildfire),NaN,7847.304610,2.156057e+06
15,21455,2020,California,California Department of Forestry and Fire Pro...,Nevada - Yuba - Placer CAL FIRE,RIOSA,12970,6/15/20 0:00,6/15/20 0:00,11 - Powerline,NaN,13.8,13.694730,1 - GPS Ground,Suppression (Wildfire),NaN,1354.188167,5.542056e+04


### Sorting Values:
Here we analyze the number of acres burned in each fired related to our specified cause.

In [117]:
my_cause.sort_values("gis_acres", ascending=False).head(3) # Sorting values - The most destructive fires caused by "my_cause"

,objectid,year_,state,agency,unit_id,fire_name,inc_num,alarm_date,cont_date,cause,comments,report_ac,gis_acres,c_method,objective,fire_num,shape_length,shape_area
20608,42055,2018,California,California Department of Forestry and Fire Pro...,Butte CAL FIRE,CAMP,16737,11/8/18 0:00,11/26/18 0:00,11 - Powerline,,153336.0,153335.56250,1 - GPS Ground,Suppression (Wildfire),NaN,311935.2788,620527017.8
21031,42478,2019,California,California Department of Forestry and Fire Pro...,Sonoma - Lake - Napa CAL FIRE,KINCADE,19376,10/23/19 0:00,11/10/19 0:00,11 - Powerline,NaN,NaN,77762.14063,1 - GPS Ground,Suppression (Wildfire),NaN,186114.8953,314692204.7
15004,36451,2002,California,California Department of Forestry and Fire Pro...,San Diego CAL FIRE,PINES,5658,7/29/02 0:00,8/11/02 0:00,11 - Powerline,NaN,61690.0,61691.23828,2 - GPS Air,Suppression (Wildfire),777,193363.8981,249655581.0


Here we are calculating the total number of acres burned due to our specified cause.

In [118]:
my_cause.gis_acres.sum() # Calculation: number of acres burned due to "my_cause"

525121.876570476

Here we are going to calculate the power line related fires over time.

In [119]:
my_cause_over_time = my_cause.groupby('year_').objectid.count().reset_index()
my_cause_over_time

,year_,objectid
0,1959,2
1,1963,2
2,1977,1
3,1984,3
4,1986,1
5,1988,1
6,1989,2
7,1990,1
8,1991,1
9,1992,5


In [120]:
powerlines_over_time = my_cause.groupby('year_').value_counts().to_frame(name = "number_of_fires").reset_index() 

In [121]:
powerlines_over_time.head(1)

,year_,objectid,state,agency,unit_id,fire_name,inc_num,alarm_date,cont_date,cause,comments,report_ac,gis_acres,c_method,objective,fire_num,shape_length,shape_area,number_of_fires
0,2006,33793,California,California Department of Forestry and Fire Pro...,Humboldt - Del Norte CAL FIRE,CASTERLIN,5546,8/18/06 0:00,8/20/06 0:00,11 - Powerline,24.2 acres grass & 9.7 acres timber,33.9,33.94157,7 - Mixed Collection Methods,Suppression (Wildfire),0,2409.321556,137356.6672,1


In [123]:
over_time_analysis = powerlines_over_time.groupby('year_').agg({'number_of_fires':'sum'}).reset_index()

In [129]:
over_time_analysis.head(20)

,year_,number_of_fires
0,2006,6
1,2007,6
2,2010,1


### Calculate Top Causes:
Now we are going to group our dataset by the column "CAUSE" and calculate the sum of acres burned per cause.

In [133]:
top_causes = ca_fires.groupby(["cause"]).gis_acres.sum().reset_index().sort_values("gis_acres", ascending=False).head(10)

In [134]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
top_causes.head(10) # Give us the top 10 Wildfires Causes.

,cause,gis_acres
5,14 - Unknown / Unidentified,14238845.14514
0,1 - Lightning,10256128.35803
17,9 - Miscellaneous,7722483.21870
15,7 - Arson,1986704.64000
10,2 - Equipment Use,1820317.96200
12,4 - Campfire,1341581.93841
1,10 - Vehicle,696625.50782
13,5 - Debris,632788.61895
2,11 - Powerline,525121.87657
11,3 - Smoking,358641.51409


### Examine the result:

In [135]:
alt.Chart(top_causes).mark_bar().encode(x = "gis_acres", y=alt.Y('cause:N', sort='-x')) # Create a chart – CAUSE/ACRES 

alt.Chart(...)

In [136]:
alt.Chart(top_causes).mark_bar().encode(x = "gis_acres", y=alt.Y('cause:N', sort='-x')).properties(title="Top Wildfire Causes in CA") # Create a title

alt.Chart(...)


## II Module: General Analysis - From 2010 to 2020 

Here we are performing the same analysis but limiting our data frame to 2010-2020.

In [137]:
ca_fires_2010 = ca_fires[ca_fires.year_ > 2010] # Filter form 2010 to 2020

### How many fires have been caused by each cause? 

In [138]:
ca_fires_2010.cause.value_counts().reset_index() # Reformat table

,index,cause
0,14 - Unknown / Unidentified,1180
1,1 - Lightning,713
2,9 - Miscellaneous,440
3,2 - Equipment Use,406
4,10 - Vehicle,235
5,11 - Powerline,190
6,7 - Arson,159
7,5 - Debris,141
8,4 - Campfire,97
9,8 - Playing with fire,37


### How many acres have been burned by each cause? 

Now we are going to group our dataset by the column "CAUSE" and calculate the sum of acres burned per cause.

In [139]:
top_causes_2010 = ca_fires_2010.groupby(["cause"]).gis_acres.sum().reset_index().sort_values("gis_acres", ascending=False).head(10)

In [140]:
top_causes_2010.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   cause      10 non-null     object 
 1   gis_acres  10 non-null     float64
dtypes: float64(1), object(1)
memory usage: 240.0+ bytes


In [141]:
top_causes_2010.head(10) # Top 10 wildfire cause between 2010 and 2020

,cause,gis_acres
0,1 - Lightning,5027323.08845
3,14 - Unknown / Unidentified,2772650.20402
14,9 - Miscellaneous,1206508.69531
1,10 - Vehicle,539022.85201
9,4 - Campfire,525163.29665
12,7 - Arson,328349.90986
2,11 - Powerline,280708.10067
7,2 - Equipment Use,161898.77883
10,5 - Debris,68728.89507
13,8 - Playing with fire,15277.16557


### Examine the result:

In [142]:
alt.Chart(top_causes_2010).mark_bar().encode(x = "gis_acres", y=alt.Y('cause:N', sort='-x')).properties(title="Top Wildfire Causes in CA between 2010 and 2020") # Create a chart with title

alt.Chart(...)

I would like to calculate top 10 causes over time.

## Calculate rate of powerline fires

For this analysis I have imported the second database: Cal Fire - Powerline Incidents.

In [145]:
powerline_fires_per_year = pd.read_csv('data/processed/powerline_fires_per_year.csv').rename(columns={'year': 'year_'}) # Import powerlines database
powerline_fires_per_year

,year_,powerline_fires_per_year
0,2011,175
1,2012,228
2,2013,248
3,2014,203
4,2015,174
5,2016,186
6,2017,274
7,2018,192
8,2019,357
9,2020,276


In [147]:
merged = fires_per_year.merge(powerline_fires_per_year) # Merge both databases
merged['pct_powerline'] = merged.powerline_fires_per_year / merged.fires_per_year * 100
merged # Here I found a big inconsistency: after merging the database with the powerlines database, I suspect that this database is highly incomplete. The number of fires caused by powerlines in 2019 excedeed the total number of fires provided in this database.

,year_,fires_per_year,powerline_fires_per_year,pct_powerline
0,2011,317,175,55.20505
1,2012,351,228,64.95726
2,2013,298,248,83.22148
3,2014,238,203,85.29412
4,2015,318,174,54.71698
5,2016,352,186,52.84091
6,2017,607,274,45.14003
7,2018,413,192,46.48910
8,2019,312,357,114.42308
9,2020,500,276,55.20000


In [148]:
powerline_fires = pd.read_csv('data/processed/powerline_fires_clean.csv'
                             )[['inc_number', 'equipment_involved_in_ignition']].rename(columns={'inc_number': 'inc_num'})
powerline_fires

,inc_num,equipment_involved_in_ignition
0,0010797,211 - Electrical power (utility) line.
1,864,211 - Electrical power (utility) line.
2,0005510,211 - Electrical power (utility) line.
3,1556,211 - Electrical power (utility) line.
4,1926,211 - Electrical power (utility) line.
...,...,...
2531,0014203,211 - Electrical power (utility) line.
2532,1361137,211 - Electrical power (utility) line.
2533,1361137,211 - Electrical power (utility) line.
2534,0166934,211 - Electrical power (utility) line.


In [149]:
all_records = ca_fires[['fire_name', 'inc_num', 'cause']].merge(powerline_fires, how='outer')
all_records

,fire_name,inc_num,cause,equipment_involved_in_ignition
0,NELSON,13212,11 - Powerline,NaN
1,AMORUSO,11799,2 - Equipment Use,NaN
2,ATHENS,18493,14 - Unknown / Unidentified,NaN
3,FLEMING,7619,9 - Miscellaneous,NaN
4,MELANESE,8471,18 - Escaped Prescribed Burn,NaN
...,...,...,...,...
23480,NaN,0014203,NaN,211 - Electrical power (utility) line.
23481,NaN,1361137,NaN,211 - Electrical power (utility) line.
23482,NaN,1361137,NaN,211 - Electrical power (utility) line.
23483,NaN,0166934,NaN,211 - Electrical power (utility) line.


In [150]:
all_records.sort_values('inc_num')

,fire_name,inc_num,cause,equipment_involved_in_ignition
18472,BUCKMAN 2,,14 - Unknown / Unidentified,NaN
18470,DEER,,14 - Unknown / Unidentified,NaN
18471,BUTTE,,1 - Lightning,NaN
18473,HILL,,1 - Lightning,NaN
8884,WOODWARDIA FIRE,0,14 - Unknown / Unidentified,NaN
...,...,...,...,...
14364,TICK,NaN,14 - Unknown / Unidentified,NaN
14365,PALERMO,NaN,14 - Unknown / Unidentified,NaN
14366,SWEETWATER,NaN,14 - Unknown / Unidentified,NaN
14367,MUREAU,NaN,NaN,NaN


In [152]:
all_records[(all_records.cause.isna() == False) & (all_records.equipment_involved_in_ignition.isna() == False)]
# This analysis also demonstrates the inconsistency of the database. Example: the same incident has different causes attributed to it. 

,fire_name,inc_num,cause,equipment_involved_in_ignition
55,DULUTH,18540,10 - Vehicle,211 - Electrical power (utility) line.
81,RESERVE,2033499,14 - Unknown / Unidentified,211 - Electrical power (utility) line.
260,BRIM,11354,9 - Miscellaneous,211 - Electrical power (utility) line.
275,MOUNTAIN,9134,11 - Powerline,211 - Electrical power (utility) line.
299,FOX,7241,4 - Campfire,211 - Electrical power (utility) line.
...,...,...,...,...
21194,MCMILLIAN,7684,2 - Equipment Use,211 - Electrical power (utility) line.
21200,GLADE,9037,2 - Equipment Use,211 - Electrical power (utility) line.
21220,R3 RYE PATCH,4067,1 - Lightning,211 - Electrical power (utility) line.
21228,CAVE,2908,7 - Arson,211 - Electrical power (utility) line.
